In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import pywt
from PIL import Image
from io import BytesIO
import base64
import pickle

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [4]:
with open("/content/drive/MyDrive/Indian Celebrity Prediction/model/celebrity_prediction_model.pickle", "rb") as file:
 model=pickle.load(file)

In [5]:
haar_cascade=cv2.CascadeClassifier('/usr/local/lib/python3.10/dist-packages/cv2/data/haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier('/usr/local/lib/python3.10/dist-packages/cv2/data/haarcascade_eye.xml')

In [6]:
def get_cropped_faces_with_2_eyes(original_image):
 gray_image=cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
 faces=haar_cascade.detectMultiScale(gray_image, 1.2, 5)
 faces_list=[]
 for x,y,w,h in faces:
  face_gray=gray_image[y:y+h, x:x+w]
  eyes=eye_cascade.detectMultiScale(face_gray)
  if len(eyes)>=2:
   face_img=original_image[y:y+h, x:x+w]
   faces_list.append(face_img)
 return faces_list

In [7]:
def image_wavelet_transform(image, mode='haar', level=1):
 image_gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 imageArray=np.float32(image_gray)/255
 coeff=pywt.wavedec2(imageArray, mode, level=level)
 coeff[0]*=0
 imageArray_wt=pywt.waverec2(coeff, mode)
 imageArray_wt=np.uint8(imageArray_wt*255)
 return imageArray_wt

In [8]:
def array_encoded_image(image_array):
 image=Image.fromarray(image_array)
 buffer=BytesIO()
 image.save(buffer, format='JPEG')
 encoded_data=base64.b64encode(buffer.getvalue()).decode('utf-8')
 encoded_image="data:image/png;base64,"+encoded_data
 return encoded_image

In [9]:
def process_uploaded_image(file):
 image_bytes=file.read()
 image_array=np.frombuffer(image_bytes, dtype=np.uint8)
 return cv2.imdecode(image_array, cv2.IMREAD_COLOR)

In [10]:
def preprocess_predict_image(file=None, image_path=None):
    if file:
        original_image=process_uploaded_image(file)
    elif image_path:
        original_image=cv2.imread(image_path)

    try:
        faces=get_cropped_faces_with_2_eyes(original_image)
    except:
        return "Invalid Image! Upload Another Image File"

    faces_img=[]
    faces_name=[]
    faces_proba=[]

    for face_img in faces:
        face_color=cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        faces_img.append(array_encoded_image(face_color))

        img_scaled=cv2.resize(face_img, (64, 64))
        img_wt=image_wavelet_transform(face_img, mode='db1', level=6)
        img_wt_scaled=cv2.resize(img_wt, (64, 64))
        combined_img=np.vstack((img_scaled.reshape(-1,1), img_wt_scaled.reshape(-1,1)))
        combined_img=combined_img.reshape(1,-1)
        combined_img=np.float32(combined_img)

        name=model.predict(combined_img)[0]
        faces_name.append(name)
        proba=model.predict_proba(combined_img).max()
        faces_proba.append(round(proba,3))

    return pd.DataFrame({"face_img": faces_img, "face_name": faces_name, "face_proba": faces_proba})

In [11]:
image_path="/content/drive/MyDrive/Indian Celebrity Prediction/datasets/images/test images/22-5.png"
results=preprocess_predict_image(image_path=image_path)
results

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but 

,face_img,face_name,face_proba
0,"data:image/png;base64,/9j/4AAQSkZJRgABAQAAAQAB...",Shah Rukh Khan,0.433
1,"data:image/png;base64,/9j/4AAQSkZJRgABAQAAAQAB...",Shah Rukh Khan,0.821
2,"data:image/png;base64,/9j/4AAQSkZJRgABAQAAAQAB...",Deepika Padukone,0.829


In [12]:
!pip install pyngrok

In [13]:
from flask import Flask, render_template, request
from pyngrok import ngrok
import json

In [14]:
app = Flask(__name__, template_folder="/content/drive/MyDrive/Indian Celebrity Prediction/templates", static_folder="/content/drive/MyDrive/Indian Celebrity Prediction/static")

In [15]:
with open("/content/drive/MyDrive/Indian Celebrity Prediction/datasets/Indian celebrities.json", 'r') as file:
 celebrities=json.loads(file.read())

In [16]:
celebrities_data=[]
for celebrity in celebrities:
 data={
  "name": celebrity,
  "img_location": f"images/{celebrity}/0.jpg"
 }
 celebrities_data.append(data)

In [17]:
@app.route('/', methods=["GET", "POST"])
def home():
    if request.method=='POST':
        file=request.files.get("image-upload")
        results=preprocess_predict_image(file)
        if type(results)==str:
            return render_template("app.html", celebrities=celebrities_data, error=results)
        else:
            return render_template("app.html", celebrities=celebrities_data, classified_faces=results.to_dict(orient="records"))
    else:
        return render_template("app.html", celebrities=celebrities_data)

In [ ]:
ngrok.set_auth_token("2gDr9vTQpw8WB4TAQ6tTkNwNSRG_4F63iNcwct5zzubERyQDE")
public_url=ngrok.connect(5000).public_url

print("click on", public_url)
app.run()

click on https://d02f-35-230-171-203.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:39] "GET /static/app.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:40] "GET /static/images/Anupam%20Kher/0.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:40] "GET /static/images/Sachin%20Tendulkar/0.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:40] "GET /static/images/Amitabh%20Bachchan/0.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:40] "GET /static/images/Hema%20Malini/0.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:40] "GET /static/images/Deepika%20Padukone/0.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2024 10:49:40] "GET /static/images/Rajkumar%20Ra